# AI4Omics Practical Session

## Task 2 - Introduction to Logistic Regression

Before we start, please, execute the following code to be ready for exercises.

- Import pandas and several scikit-learn modules that will be used in the exercises:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics

- Define the function `calculate_accuracy`:

In [ ]:
def calculate_accuracy(classifier, X_train, X_test, y_train, y_test):
    classifier.fit(X_train, y_train)
    y_pred_train = classifier.predict(X_train)
    y_pred_test = classifier.predict(X_test)
    accuracy_train = metrics.accuracy_score(y_train, y_pred_train)
    accuracy_test = metrics.accuracy_score(y_test, y_pred_test)
    print('Train accuracy:', '{:.3f}'.format(accuracy_train), 'Test accuracy:', '{:.3f}'.format(accuracy_test))
    return accuracy_train, accuracy_test, classifier

- Import data and create targets **y**:

In [ ]:
random_state = 42
data = pd.read_csv('../data/colon_cancer.csv', sep=';', index_col='id_sample')
print('data', data.shape)
y = data['tissue_status']
print('y', y.shape)

## Question 1. Create a dataframe X including all the available features (genes)

The original dataset **data** contains 60 columns with the expression levels of 60 genes and one column `tissue_status` with the sample types (normal or tumoral).

- Extract 60 columns corresponding to gene expression levels from **data** to a separate dataframe **X**.

Hint: you can use one of the methods `select_dtypes('number')` or `drop(columns=['tissue_status'])`. Please check the documentation of pandas for these two methods if you are not familiar with them.

In [ ]:
# X = data... # to complete

## Question 2. Create train and test datasets with 3/4 and 1/4 of samples respectively

- Create a training dataset **X_train** and a test dataset **X_test** with their corresponding targets **y_train** and **y_test**.

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=..., random_state=random_state, stratify=y) # test_size to define

- How many samples contain the obtained train and test datasets?

Hint: you can use `shape` attribute of the dataframe.

In [ ]:
# print('Train:', X_train..., 'Test:', X_test...) # to complete

## Question 3. Perform a standardisation of data

*Data standardisation* is a typical and often mandatory step in a machine learning pipeline. Data standardisation is a *feature scaling* technique aiming to convert original data, where multiple features can be spanning varying ranges and degrees of magnitude, into a comparable range of values. The data standardisation significantly improves the performance of many machine learning algorithms.

The most common standardisation technique is *Z-score* (or standard score) where the values are centered around the mean with a unit standard deviation.  

**Warning!** The calculation of the mean $\mu$ and the standard deviation $\sigma$ must be performed **on the training dataset** only. The test dataset should not be used in the calculation. It will be scaled using the values $\mu$ and $\sigma$ obtained in the training dataset.

In `scikit-learn`, data standardisation can be realized with a `StandardScaler` object. 

- Execute the following code to calculate $\mu$ and $\sigma$ for **X_train** dataset.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # create a scaler
scaler.fit(X_train) # calculate mu and sigma on X_train (only training dataset should be used!)

- Display $\mu$ (mean) values

In [ ]:
# Display calculated mu (mean) for each feature
print('Mean mu', scaler.mean_)

- Display $\sigma$ (standard deviation) values

In [ ]:
# Display calculated sigma (standard deviation) for each feature
print('Std sigma', scaler.scale_)

Now, we can use $\mu$ and $\sigma$ calculated from **X_train** dataset to perform a standardisation of both **X_train** and **X_test**. Use the `transform` method of the `scaler`object. As a result, you should obtain two scaled datasets: **X_train_scaled** and **X_test_scaled**.

- Execute the code for **X_train**:

In [ ]:
# To execute
X_train_scaled = scaler.transform(X_train) # numpy object
X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns) # convert to pandas DataFrame format

- Complete a similar code for **X_test** and execute it:

In [ ]:
# X_test_scaled = scaler.transform(...) # to complete

# X_test_scaled = pd.DataFrame(X_test_scaled, index=...index, columns=...columns) # to complete

After the standardisation, the mean values of expression levels should be equal to 0 for all genes in **X_train_scaled**, and the standard deviation should be equal to 1.

- Check that the mean values are 0 for the first 5 features in **X_train_scaled**

In [ ]:
# X_train_scaled... # to complete

- Check that the standard deviations are 1 for the first 5 features in **X_train_scaled**

In [ ]:
# X_train_scaled... # to complete

## Question 4. Create a model of Logistic Regression (LR)

Logistic regression uses an analytical function, called *logistic function* or *sigmoid function*, which has a characteristic S-shape. By optimizing the coefficients of this function (max likelihood or min cross-entropy), it makes it possible to estimate the probability for a sample to belong to this or that class. For example, tumoral versus normal.

<img src="logistic_regression.png" alt="Logistic regression" width="600" aling="center">

- Create a Logistic Regression classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=random_state, penalty='none')
print(classifier)

- Train the classifier and calculate its accuracy using `calculate_accuracy` function

In [ ]:
# accuracy_train, accuracy_test, trained_classifier = calculate_accuracy(classifier, ... , ... , y_train, y_test) # to complete

## Question 5. Display a confusion matrix for LR algorithm

A *confusion matrix* is a table that allows visualisation of the performance of a supervised algorithm. Each row of the matrix represents the instances in an actual class while each column represents the instances in a predicted class.

- Display a confusion maxtrix for LR algorithm **using the test dataset only** 

In [ ]:
# metrics.plot_confusion_matrix(trained_classifier, ... , ...) # to complete

## Question 6. Evaluate the impact of each gene in LR classifier

After the training phase, it is possible to know the coefficients $\beta$ of LR model for each feature (gene). They are available in the `coef_` attribute. The greater is the coefficient $\beta$ (in absolute value), the greater is the impact of the corresponding gene in the model. By analysing the coefficients $\beta$ of the trained LR model, we can find the most predictive genes.

- Display the coefficients $\beta$ for the first 5 features.

In [ ]:
coefficients = pd.DataFrame(trained_classifier.coef_[0], index=X_train_scaled.columns, columns=['beta'])
coefficients.head()

- Display the coefficients $\beta$ as a *barplot* from the smallest to the biggest value.

In [ ]:
coefficients = coefficients.sort_values(by='beta')
coefficients.plot.bar(figsize=(15, 3))

- What genes have the most important impact in the model?

- What genes have almost no impact on the prediction? 

- Select the best 3 features mostly impacting the model

In [ ]:
n_features = 3
coefficients['abs_beta'] = coefficients['beta'].abs() # calculate the absolute values of betas
coefficients = coefficients.sort_values(by='abs_beta', ascending=False) # sort by arsolute values
top_features = list(coefficients.head(n_features).index) # list of N top features
print('Top features of LR:', top_features)

## Question 7. Calculate the performance of LR model using 1, 2, ... N top features

- What does the code below calculate? Execute it and explain the result. 

In [ ]:
for i in range(len(top_features)):
    selected_features = top_features[0:i+1]
    print(selected_features)
    accuracy_train, accuracy_test, trained_classifier = calculate_accuracy(classifier, X_train_scaled[selected_features], X_test_scaled[selected_features], y_train, y_test)

- Do we need all the 60 genes in the model? What do you think? If we can reduce the number of features, how many should we keep? 

## Question 8. Case study

The *AI-Hospital* has developed a new diagnostic tool for colon cancer based on the expression levels of a panel of 3 genes. This tool produced the following measurements for a new patient arrived in the hospital:

In [ ]:
panel = ['RNF43', 'SLC7A5', 'DAO']

In [ ]:
new_patient = {'RNF43': 4.68, 'SLC7A5': 4.10, 'DAO': 7.59}

- **Does this patient have a colon cancer?**

Hint: To answer this question, train a LR model on the totality of available data **X**. In this case, **X_train** will contain all the samples of **X**. **X_test** will have only one sample corresponding to the new patient. Do not forget to scale the data properly, `fit` on **X_train** only and then  `transform` on both **X_train** and **X_test**.

In [ ]:
X_train = data[panel]
y_train = y
X_train.head(3)

In [ ]:
X_test = pd.DataFrame([new_patient], index=['new_patient'])
X_test

In [ ]:
# Code to write by yourself